In [4]:
# Import
import pandas as pd
import sys
from datetime import datetime
sys.path.append('../..')
import pickle

from Classifier.data_processing import processor

In [5]:
start_time = datetime(2017,3,1)
end_time = datetime(2018,2,28)
interval = 1440 # 1440 minutes = 1 day

data, mean, std = processor.historical_download(start_time, end_time, interval, normalize=True)

No Reddit pickle found


In [6]:
# Split out 1 month validation data
train_data = data[:-30]
valid_data = data[-30:]

In [8]:
print(valid_data.head(1))
print(valid_data.tail(1))

            Btc_addresses  Btc_daily_trx  Btc_fee_per_trx  Btc_hashrate  \
2018-01-24       0.320864      -0.569948         0.087758     -0.140748   

            Btc_search_DE  Btc_search_FR  Btc_search_GB  Btc_search_KR  \
2018-01-24      -0.455087      -0.295326      -0.530261            NaN   

            Btc_search_RU  Btc_search_US        ...          \
2018-01-24       -0.13814      -0.237623        ...           

            Ltcusd_gdax_close  Ltcusd_gdax_high  Ltcusd_gdax_low  \
2018-01-24          -0.015969          -0.19435         0.270428   

            Ltcusd_gdax_open  Ltcusd_gdax_vol  Ltcusd_kraken_close  \
2018-01-24         -0.217916        -0.426414            -0.265307   

            Ltcusd_kraken_high  Ltcusd_kraken_low  Ltcusd_kraken_open  \
2018-01-24           -0.247469           0.263901           -0.016245   

            Ltcusd_kraken_vol  
2018-01-24          -0.500546  

[1 rows x 96 columns]
            Btc_addresses  Btc_daily_trx  Btc_fee_per_trx  Bt

In [5]:
train_x, train_y = processor.generate_x_y(train_data, target="Btceur_kraken_close") 